In [ ]:
from nav2_simple_commander.robot_navigator import BasicNavigator

import rclpy as rp
from rclpy.node import Node
from rclpy.executors import MultiThreadedExecutor
from rclpy.duration import Duration

import tf2_ros
# from geometry_msgs.msg import PoseStamped
from geometry_msgs.msg import TransformStamped
from visualization_msgs.msg import MarkerArray

import time
import cv2
# import rclpy as rp
import numpy as np
import logging

# from rclpy.node import Node
from sensor_msgs.msg import Image
from geometry_msgs.msg import PoseStamped, PoseArray
from rcl_interfaces.msg import ParameterDescriptor, ParameterType
from visualization_msgs.msg import Marker, MarkerArray
from cv_bridge import CvBridge
from ament_index_python.packages import get_package_share_directory
from tf_transformations import quaternion_from_matrix

In [ ]:
import rclpy as rp
import numpy as np
import os

from ultralytics import YOLO
from rclpy.node import Node
from sensor_msgs.msg import Image
from cv_bridge import CvBridge

from yolo_inference.msg import InferenceResult
from yolo_inference.msg import Yolov8Inference

from ament_index_python.packages import get_package_share_directory

class ObjectDetection(Node):
    def __init__(self):

        super().__init__('object_dectection_node')
        self.cv_bridge = CvBridge()
        self.model_name_param = self.declare_parameter('model_name', 'yolov8n.pt')
        self.pkg_share_path = get_package_share_directory('yolo_pkg')
        self.model_dir = os.path.join(self.pkg_share_path, 'models')
        self.model_name = self.model_name_param.value
        self.model_path = os.path.join(self.model_dir, self.model_name)
        self.model = YOLO(self.model_path)
        self.get_logger().info(f"model : {self.model_path}")

        self.yolov8_inference = Yolov8Inference()

        self.camera_sub = self.create_subscription(
            Image,
            'image_raw',
            self.camera_callback,
            10
        )
        self.camera_sub

        self.yolov8_pub = self.create_publisher(Yolov8Inference, "/yolov8_inference", 1)
        self.img_pub = self.create_publisher(Image, "/inference_result", 1)
        
        self.img_size = 0
        self.is_detected = False



    def camera_callback(self, msg):
        self.is_detected = False

        img = self.cv_bridge.imgmsg_to_cv2(msg, "bgr8")
        results = self.model(img)
        
        if self.img_size == 0:
            self.img_size = msg.height * msg.width

        self.yolov8_inference.header.frame_id = "inference"
        self.yolov8_inference.header.stamp = self.get_clock().now().to_msg()

        for result in results:
            boxes = result.boxes.data.cpu()
            # box = np.array(boxes)
            for box in boxes:
                self.inference_result = InferenceResult()
                label = int(box[5])
                self.inference_result.label = self.model.names[label]
                self.inference_result.top = int(box[0])
                self.inference_result.top_left = int(box[1])
                self.inference_result.bottom = int(box[2])
                self.inference_result.bottom_right = int(box[3])
                self.inference_result.conf = float(box[4])
                self.yolov8_inference.yolov8_inference.append(self.inference_result)

                each_box_size = (self.inference_result.bottom - self.inference_result.top) * (self.inference_result.bottom_right - self.inference_result.top_left)
                print(f"each_box_size: {each_box_size}")
                print(f"img_size: {self.img_size}")
                print("self.inference_result.label type:", type(self.inference_result.label))
                print(self.inference_result.label)
                if each_box_size / self.img_size >= 0.5 and self.inference_result.label == "person":
                    self.is_detected = True

        annotated_frame = results[0].plot()
        img_msg = self.cv_bridge.cv2_to_imgmsg(annotated_frame)

#         print("img_msg in ObjectionDetection: ")
#         print(img_msg)
#         print("self.yolov8_inference in ObjectionDetection: ")
#         print(self.yolov8_inference)

        self.img_pub.publish(img_msg)
        self.yolov8_pub.publish(self.yolov8_inference)
        self.yolov8_inference.yolov8_inference.clear()


In [ ]:
class Stack:
    def __init__(self):
        self.items = []

    def is_empty(self):
        return len(self.items) == 0

    def push(self, item):
        self.items.append(item)

    def pop(self):
        if not self.is_empty():
            return self.items.pop()

    def top(self):
        if not self.is_empty():
            return self.items[-1]

    def size(self):
        return len(self.items)


In [ ]:
pose_stack = Stack()
real_map = [
    [ 0 , 'D', 'D'],
    ['A', 'B', 'C']
]
visited = [
    [0, 0, 0],
    [0, 0, 0]
]


rp.init()

nav = BasicNavigator()

# position A (initial pose)
pos_a = PoseStamped()
pos_a.header.frame_id = 'map'
pos_a.header.stamp = nav.get_clock().now().to_msg()
pos_a.pose.position.x = 0.0
pos_a.pose.position.y = 0.0
pos_a.pose.orientation.z = 0.0
pos_a.pose.orientation.w = 0.0
nav.setInitialPose(pos_a)

pose_stack.push(pos_a)

nav.waitUntilNav2Active()

In [ ]:
# rp.init()

od = ObjectDetection()

# try:
#     while True:
#         rp.spin_once(od)
#         if (od.is_detected == True):
#             print("Is detected!!!!!!")
# #             nav.cancelTask()
#             raise Exception("Test")
# except Exception as e:
#     print(e)

In [ ]:
# position B
pos_b = PoseStamped()
pos_b.header.frame_id = 'map'
pos_b.header.stamp = nav.get_clock().now().to_msg()
pos_b.pose.position.x = 1.0
pos_b.pose.position.y = 0.3
pos_b.pose.orientation.w = 0.
pos_b.pose.orientation.z = 0.

# position C
pos_c = PoseStamped()
pos_c.header.frame_id = 'map'
pos_c.header.stamp = nav.get_clock().now().to_msg()
pos_c.pose.position.x = 1.7
pos_c.pose.position.y = 0.
pos_c.pose.orientation.w = 0.
pos_c.pose.orientation.z = 0.

# position D
pos_d = PoseStamped()
pos_d.header.frame_id = 'map'
pos_d.header.stamp = nav.get_clock().now().to_msg()
pos_d.pose.position.x = 1.6
pos_d.pose.position.y = 1.0
pos_d.pose.orientation.w = 0.
pos_d.pose.orientation.z = 0.

In [ ]:
def move_to_goal(goal_pos):
    
    goal_pos.header.stamp = nav.get_clock().now().to_msg()
    
    nav.goToPose(goal_pos)

    i = 0
    while not nav.isTaskComplete():
        i = i + 1

        rp.spin_once(od)
        if (od.is_detected == True):
            print("Is detected!!!!!!")
            nav.cancelTask()
            nav.spin(3.14)
            time.sleep(8)
            if pose_stack.is_empty() == False:
                move_to_goal(pose_stack.top())
            if pose_stack.size() > 1:
                pose_stack.pop()
                move_to_goal(pose_stack.top())
            raise Exception("Object Detection!")
        
        feedback = nav.getFeedback()
        if feedback and i % 5 == 0:
            print('Distance remaining: ' + '{:.2f}'.format(
            feedback.distance_remaining) + 'meters.')
#             print(feedback.navigation_time)

            if feedback.distance_remaining <= 0.25 and Duration.from_msg(feedback.navigation_time) > Duration(seconds=5.0):
                nav.cancelTask()
    if nav.isTaskComplete():
        print("nav2 task completed.")

In [ ]:
def move_with_short_delay(arg_x, arg_y, arg_w, arg_z):
    
    print(f"Move to pose: x {arg_x}, y {arg_y}, orientation: w {arg_w}, z {arg_z}")
    
    goal_pose1 = PoseStamped()
    goal_pose1.header.frame_id = 'map'
    goal_pose1.header.stamp = nav.get_clock().now().to_msg()
    goal_pose1.pose.position.x = arg_x
    goal_pose1.pose.position.y = arg_y
    goal_pose1.pose.orientation.w = arg_w
    goal_pose1.pose.orientation.z = arg_z

    nav.goToPose(goal_pose1)

    i = 0
    while not nav.isTaskComplete():
        i = i + 1

#         rp.spin_once(od)
#         if (od.is_detected == True):
#             print("Is detected!!!!!!")
#             nav.cancelTask()
#             throw Exception("Test")
        
        feedback = nav.getFeedback()
        if feedback and i % 5 == 0:
            print('Distance remaining: ' + '{:.2f}'.format(
            feedback.distance_remaining) + 'meters.')
#             print(feedback.navigation_time)

            if feedback.distance_remaining < 0.3 and Duration.from_msg(feedback.navigation_time) > Duration(seconds=2.0):
                nav.cancelTask()
            

In [ ]:
def task1():
    print("[Task 1]")
    print("move to B")
    move_to_goal(pos_b)
    print("spin 90")
    time.sleep(2)
    nav.spin(-1.57)
    time.sleep(6)
    pose_stack.push(pos_b)

def task2():
    print("[Task 2]")
    print("move to C")
    move_to_goal(pos_c)
    print("spin 360")
    time.sleep(2)
    nav.spin(6.28)
    time.sleep(10)
    pose_stack.push(pos_c)

def task3():
    print("move to D")
    move_to_goal(pos_d)
    print("The End...")
    pose_stack.push(pos_d)

In [ ]:
pos_0 = PoseStamped()
pos_0.header.frame_id = 'map'
pos_0.header.stamp = nav.get_clock().now().to_msg()
pos_0.pose.position.x = 0.35
pos_0.pose.position.y = 0.
pos_0.pose.orientation.w = 0.
pos_0.pose.orientation.z = 0.
move_to_goal(pos_0)

In [ ]:
while True:
    try:
        task1()
    except Exception as e:
        print(e)
        continue
    break

while True:
    try:
        task2()
    except Exception as e:
        print(e)
        continue
    break


while True:
    try:
        task3()
    except Exception as e:
        print(e)
        continue
    break

In [ ]:
def move_with_long_delay(arg_x, arg_y, arg_w, arg_z):
    
    print(f"Move to pose: x {arg_x}, y {arg_y}, orientation: w {arg_w}, z {arg_z}")
    
    goal_pose1 = PoseStamped()
    goal_pose1.header.frame_id = 'map'
    goal_pose1.header.stamp = nav.get_clock().now().to_msg()
    goal_pose1.pose.position.x = arg_x
    goal_pose1.pose.position.y = arg_y
    goal_pose1.pose.orientation.w = arg_w
    goal_pose1.pose.orientation.z = arg_z

    nav.goToPose(goal_pose1)

    i = 0
    while not nav.isTaskComplete():
        i = i + 1

        rp.spin_once(od)
        if (od.is_detected == True):
            print("Is detected!!!!!!")
            nav.cancelTask()
            raise Exception("Test")
        
        feedback = nav.getFeedback()
        if feedback and i % 5 == 0:
            print('Distance remaining: ' + '{:.2f}'.format(
            feedback.distance_remaining) + 'meters.')
#             print(feedback.navigation_time)

            if feedback.distance_remaining < 0.2 and Duration.from_msg(feedback.navigation_time) > Duration(seconds=5.0):
                nav.cancelTask()

In [ ]:
nav.spin(6.28)
time.sleep(4)
move_with_long_delay(1.5, 1., 1., 1.) # to C

In [ ]:
nav.spin(1.57)
time.sleep(2)
move_with_long_delay(0., 0., 0., 0.)

In [ ]:
move_with_long_delay(0., 0., 0., 0.)

In [ ]:
nav.backup(0.15, 0.25)

In [ ]:

# position X
pos_x = PoseStamped()
pos_x.header.frame_id = 'map'
pos_x.header.stamp = nav.get_clock().now().to_msg()
pos_x.pose.position.x = 0.0
pos_x.pose.position.y = 0.0
pos_x.pose.orientation.w = 0.
pos_x.pose.orientation.z = 0.


try:
    move_to_goal(pos_x)
except Exception as e:
    print(e)

In [ ]:
time.sleep(2)
nav.backup(0.25, 0.5)
nav.spin(-0.77)
time.sleep(4)
nav.backup(0.25, 0.5)

In [ ]:
#! /usr/bin/env python3
# Copyright 2021 Samsung Research America
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from geometry_msgs.msg import PoseStamped
from nav2_simple_commander.robot_navigator import BasicNavigator, TaskResult
import rclpy
from rclpy.duration import Duration

"""
Basic navigation demo to go to pose.
"""


def main():
    rclpy.init()

    navigator = BasicNavigator()

    # Set our demo's initial pose
    initial_pose = PoseStamped()
    initial_pose.header.frame_id = 'map'
    initial_pose.header.stamp = navigator.get_clock().now().to_msg()
    initial_pose.pose.position.x = 0.0
    initial_pose.pose.position.y = 0.0
    initial_pose.pose.orientation.z = 0.0
    initial_pose.pose.orientation.w = 0.0
    navigator.setInitialPose(initial_pose)

    # Activate navigation, if not autostarted. This should be called after setInitialPose()
    # or this will initialize at the origin of the map and update the costmap with bogus readings.
    # If autostart, you should `waitUntilNav2Active()` instead.
    # navigator.lifecycleStartup()

    # Wait for navigation to fully activate, since autostarting nav2
    navigator.waitUntilNav2Active()

    # If desired, you can change or load the map as well
    # navigator.changeMap('/path/to/map.yaml')

    # You may use the navigator to clear or obtain costmaps
    # navigator.clearAllCostmaps()  # also have clearLocalCostmap() and clearGlobalCostmap()
    # global_costmap = navigator.getGlobalCostmap()
    # local_costmap = navigator.getLocalCostmap()

    # Go to our demos first goal pose
    goal_pose = PoseStamped()
    goal_pose.header.frame_id = 'map'
    goal_pose.header.stamp = navigator.get_clock().now().to_msg()
    goal_pose.pose.position.x = 1.0
    goal_pose.pose.position.y = 0.5
    goal_pose.pose.orientation.w = 1.0

    # sanity check a valid path exists
    # path = navigator.getPath(initial_pose, goal_pose)

    navigator.goToPose(goal_pose)

    i = 0
    while not navigator.isTaskComplete():
        ################################################
        #
        # Implement some code here for your application!
        #
        ################################################

        # Do something with the feedback
        i = i + 1
        feedback = navigator.getFeedback()
        if feedback and i % 5 == 0:
            print('Estimated time of arrival: ' + '{0:.0f}'.format(
                  Duration.from_msg(feedback.estimated_time_remaining).nanoseconds / 1e9)
                  + ' seconds.')

            # Some navigation timeout to demo cancellation
            if Duration.from_msg(feedback.navigation_time) > Duration(seconds=600.0):
                navigator.cancelTask()

            # Some navigation request change to demo preemption
            if Duration.from_msg(feedback.navigation_time) > Duration(seconds=18.0):
                goal_pose.pose.position.x = 1.5
                goal_pose.pose.position.x = 1.0
                navigator.goToPose(goal_pose)

    # Do something depending on the return code
    result = navigator.getResult()
    if result == TaskResult.SUCCEEDED:
        print('Goal succeeded!')
    elif result == TaskResult.CANCELED:
        print('Goal was canceled!')
    elif result == TaskResult.FAILED:
        print('Goal failed!')
    else:
        print('Goal has an invalid return status!')

    navigator.lifecycleShutdown()

    exit(0)

main()

In [ ]:
nav = navigator

In [ ]:
def move_with_long_delay(arg_x, arg_y, arg_w, arg_z):
    
    print(f"Move to pose: x {arg_x}, y {arg_y}, orientation: w {arg_w}, z {arg_z}")
    
    goal_pose1 = PoseStamped()
    goal_pose1.header.frame_id = 'map'
    goal_pose1.header.stamp = nav.get_clock().now().to_msg()
    goal_pose1.pose.position.x = arg_x
    goal_pose1.pose.position.y = arg_y
    goal_pose1.pose.orientation.w = arg_w
    goal_pose1.pose.orientation.z = arg_z

    nav.goToPose(goal_pose1)

    i = 0
    while not nav.isTaskComplete():
        i = i + 1

        rp.spin_once(od)
        if (od.is_detected == True):
            print("Is detected!!!!!!")
            nav.cancelTask()
            throw Exception("Test")
        
        feedback = nav.getFeedback()
        if feedback and i % 5 == 0:
            print('Distance remaining: ' + '{:.2f}'.format(
            feedback.distance_remaining) + 'meters.')
#             print(feedback.navigation_time)

            if feedback.distance_remaining < 0.3 and Duration.from_msg(feedback.navigation_time) > Duration(seconds=5.0):
                nav.cancelTask()
            